# Install dependencies

In [ ]:
%pip install -U -r requirements.txt

# Import SageMaker Defaults Configurations

The Amazon SageMaker Python SDK supports setting of default values for AWS infrastructure primitive types, such as instance types, Amazon S3 folder locations, and IAM roles. You can override the default locations of these files by setting the `SAGEMAKER_USER_CONFIG_OVERRIDE` environment variables for the user-defined configuration file paths.

In [ ]:
import os

# Use the current working directory as the location for SageMaker Python SDK config file
os.environ["SAGEMAKER_USER_CONFIG_OVERRIDE"] = os.getcwd()

# Download dataset

Download the dataset from the UCI website.

In [ ]:
import urllib
import os

input_data_dir = 'data/'
if not os.path.exists(input_data_dir):
    os.makedirs(input_data_dir)
input_data_path = os.path.join(input_data_dir, 'predictive_maintenance_raw_data_header.csv')
dataset_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00601/ai4i2020.csv"
urllib.request.urlretrieve(dataset_url, input_data_path)

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv(input_data_path)

print('The shape of the dataset is:', df.shape)

# Test case 1: Deploy SageMaker Endpoint

## Expected result: The user should not be able to deploy an endpoint

### Important! Make sure to execute [01_SageMaker_Jobs.ipynb](./01_SageMaker_Jobs.ipynb)

In [ ]:
import time
import os
import sagemaker
from sagemaker import get_execution_role, session
import boto3

In [ ]:
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name

role = get_execution_role()
s3_bucket_name = sagemaker_session.default_bucket()

sm_client = boto3.client('sagemaker', region_name=region)

Utility function for getting the last succeed job name

In [ ]:
job_prefix = "amzn-sm-btd-train"

In [ ]:
def get_last_job_name(job_name_prefix):
    import boto3
    sagemaker_client = boto3.client('sagemaker')

    search_response = sagemaker_client.search(
        Resource='TrainingJob',
        SearchExpression={
            'Filters': [
                {
                    'Name': 'TrainingJobName',
                    'Operator': 'Contains',
                    'Value': job_name_prefix
                },
                {
                    'Name': 'TrainingJobStatus',
                    'Operator': 'Equals',
                    'Value': "Completed"
                }
            ]
        },
        SortBy='CreationTime',
        SortOrder='Descending',
        MaxResults=1)

    return search_response['Results'][0]['TrainingJob']['TrainingJobName']

In [ ]:
job_name = get_last_job_name(job_prefix)

job_name

Create model package specifications

In [ ]:
from sagemaker.image_uris import retrieve

In [ ]:
image_uri = retrieve("xgboost", region=region, version="latest", image_scope="inference")

image_uri

In [ ]:
from sagemaker.model import Model

In [ ]:
# Specify the model source
model_url = f"s3://{s3_bucket_name}/{job_name}/{job_name}/output/model.tar.gz"

model = Model(image_uri=image_uri, model_data=model_url, role=role)

model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=f"{job_prefix}-endpoint"
)